In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

In [2]:
df = pd.read_csv('C:\\Users\\Ufuk Altan\\Downloads\\Downloads\\Data Science\\I Dati\\Prosperity.csv')
df.head()

,Country,GDP Per Capita,Technology,Human Capital,Institutional Framework,Sustainable Resources,Demand Environment,Education,Happiness,Unemployment Rate,Population,Import Degree,Import Eigenvector,Import Betweenness,Export Degree,Export Eigenvector,Export Betweenness,Location
0,Albania,5260.9,3.40,4.62,4.70,6.19,3.16,1153,4.719,0.120,2880248,0,0.000000,0.000000,0,0.000000,0.000000,Europe
1,Algeria,4449.8,2.97,3.58,3.60,5.36,5.00,1086,5.211,0.117,43216262,3,0.022113,0.000000,3,0.029143,0.000000,Africa
2,Argentina,10667.1,3.78,4.94,4.16,5.84,4.99,1187,6.086,0.101,44866075,6,0.043574,0.000000,4,0.038238,0.000000,South America
3,Armenia,4190.2,3.53,5.07,4.93,5.85,3.68,1345,4.559,0.169,2958996,0,0.000000,0.000000,0,0.000000,0.000000,Middle East
4,Australia,56698.1,6.91,7.41,8.32,6.47,5.78,1556,7.228,0.052,25265536,20,0.098483,0.003389,21,0.129649,0.003652,Oceania


In [3]:
def percentage_error(dataframe,pred, true_value):
    
    '''
    dataframe: Pass in pandas dataframe variable
    pred = 'Predictions' #model.predict(x_test).reshape()
    true_value = 'True Values' #y_test
    
    '''
    
    difference = dataframe[true_value] - dataframe[pred]
    
    dataframe['Error(%)']= np.absolute(difference/dataframe[true_value]*100)
    
    print('median \t {}'.format(dataframe['Error(%)'].median()))    
    print(dataframe['Error(%)'].describe())
    
def adjusted_r_squared(r2, data):
    
    x = (1-r2)*(data.shape[0]-1)/(data.shape[0]-data.shape[1]-1) 
    
    return 1-x

In [4]:
cols = ['Technology', 'Human Capital',
       'Institutional Framework', 'Sustainable Resources',
       'Demand Environment', 'Education', 'Happiness', 'Unemployment Rate',
       'Population', 
       'Export Eigenvector',
       'Location',
        'GDP Per Capita']

df = df[cols]

In [5]:
X = df.drop(['GDP Per Capita', 'Technology'], axis =1)
y = df[['GDP Per Capita', 'Technology']]

In [6]:
X = pd.get_dummies(X)
X.drop('Location_Europe', inplace = True, axis =1)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.15 ) 

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor

xgbRegressor = XGBRegressor(objective="reg:linear", random_state=0) #"reg:squarederror"

model = MultiOutputRegressor(xgbRegressor)
model.fit(X_train, y_train)

[12:41:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[12:41:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[12:41:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[12:41:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.


MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, gamma=None,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_depth=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            objective='reg:linear',
                                            rando

In [11]:
y_pred = model.predict(X_test)

In [20]:
y_pred[:,0]

array([ 4626.861 , 74793.94  , 17780.947 , 24063.389 , 15580.574 ,
       10679.232 , 58140.434 , 18128.352 ,  4871.7095, 59072.71  ,
       16085.807 ,  1417.556 ,  2452.3887,  8308.295 , 23573.836 ],
      dtype=float32)

In [19]:
y_test.values[:,0]

array([ 6301.1, 53867.2, 10667.1, 18856.9, 12189.5,  2964.2, 46733. ,
       23175.8, 12453.7, 42260.9, 17634.4,   717.5,  1485.4,  9080.3,
       26531.8])

In [21]:
test0 = pd.DataFrame()
test0['TrueValues'] = y_test.values[:,0]
test0['Predictions'] = y_pred[:,0]

In [22]:
percentage_error(test0, 'Predictions', 'TrueValues')

median 	 27.819633444768037
count     15.000000
mean      52.384314
std       62.775220
min        8.501978
25%       23.094382
50%       27.819633
75%       62.990488
max      260.273680
Name: Error(%), dtype: float64


In [23]:
test1 = pd.DataFrame()
test1['TrueValues'] = y_test.values[:,1]
test1['Predictions'] = y_pred[:,1]

In [24]:
percentage_error(test1, 'Predictions', 'TrueValues')

median 	 5.743398122001139
count    15.000000
mean      7.624738
std       7.606796
min       0.904582
25%       2.364778
50%       5.743398
75%       9.485995
max      27.103200
Name: Error(%), dtype: float64
